# Chapter 2: Reading and Writing Files

## Reading and writing CSV files

### Getting ready

In [1]:
import polars as pl

### How to do it...

In [28]:
df = pl.read_csv('../data/customer_shopping_data.csv')
df.head()

invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
str,str,str,i64,str,i64,f64,str,str,str
"""I138884""","""C241288""","""Female""",28,"""Clothing""",5,1500.4,"""Credit Card""","""5/8/2022""","""Kanyon"""
"""I317333""","""C111565""","""Male""",21,"""Shoes""",3,1800.51,"""Debit Card""","""12/12/2021""","""Forum Istanbul…"
"""I127801""","""C266599""","""Male""",20,"""Clothing""",1,300.08,"""Cash""","""9/11/2021""","""Metrocity"""
"""I173702""","""C988172""","""Female""",66,"""Shoes""",5,3000.85,"""Credit Card""","""16/05/2021""","""Metropol AVM"""
"""I337046""","""C189076""","""Female""",53,"""Books""",4,60.6,"""Cash""","""24/10/2021""","""Kanyon"""


In [24]:
df = pl.read_csv('../data/customer_shopping_data_no_header.csv')
df.head()

I138884,C241288,Female,28,Clothing,5,1500.4,Credit Card,5/8/2022,Kanyon
str,str,str,i64,str,i64,f64,str,str,str
"""I317333""","""C111565""","""Male""",21,"""Shoes""",3,1800.51,"""Debit Card""","""12/12/2021""","""Forum Istanbul…"
"""I127801""","""C266599""","""Male""",20,"""Clothing""",1,300.08,"""Cash""","""9/11/2021""","""Metrocity"""
"""I173702""","""C988172""","""Female""",66,"""Shoes""",5,3000.85,"""Credit Card""","""16/05/2021""","""Metropol AVM"""
"""I337046""","""C189076""","""Female""",53,"""Books""",4,60.6,"""Cash""","""24/10/2021""","""Kanyon"""
"""I227836""","""C657758""","""Female""",28,"""Clothing""",5,1500.4,"""Credit Card""","""24/05/2022""","""Forum Istanbul…"


In [25]:
df = pl.read_csv('../data/customer_shopping_data_no_header.csv', has_header=False)
df.head()

column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10
str,str,str,i64,str,i64,f64,str,str,str
"""I138884""","""C241288""","""Female""",28,"""Clothing""",5,1500.4,"""Credit Card""","""5/8/2022""","""Kanyon"""
"""I317333""","""C111565""","""Male""",21,"""Shoes""",3,1800.51,"""Debit Card""","""12/12/2021""","""Forum Istanbul…"
"""I127801""","""C266599""","""Male""",20,"""Clothing""",1,300.08,"""Cash""","""9/11/2021""","""Metrocity"""
"""I173702""","""C988172""","""Female""",66,"""Shoes""",5,3000.85,"""Credit Card""","""16/05/2021""","""Metropol AVM"""
"""I337046""","""C189076""","""Female""",53,"""Books""",4,60.6,"""Cash""","""24/10/2021""","""Kanyon"""


In [30]:
column_names = ['invoice_no', 'customer_id', 'gender', 'age', 'category', 'quantity', 'price', 'payment_method', 'invoice_date', 'shopping_mall']
df = pl.read_csv('../data/customer_shopping_data_no_header.csv', 
                 has_header=False, 
                 new_columns=column_names)
df.head()

invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
str,str,str,i64,str,i64,f64,str,str,str
"""I138884""","""C241288""","""Female""",28,"""Clothing""",5,1500.4,"""Credit Card""","""5/8/2022""","""Kanyon"""
"""I317333""","""C111565""","""Male""",21,"""Shoes""",3,1800.51,"""Debit Card""","""12/12/2021""","""Forum Istanbul…"
"""I127801""","""C266599""","""Male""",20,"""Clothing""",1,300.08,"""Cash""","""9/11/2021""","""Metrocity"""
"""I173702""","""C988172""","""Female""",66,"""Shoes""",5,3000.85,"""Credit Card""","""16/05/2021""","""Metropol AVM"""
"""I337046""","""C189076""","""Female""",53,"""Books""",4,60.6,"""Cash""","""24/10/2021""","""Kanyon"""


In [31]:
df = pl.read_csv('../data/customer_shopping_data_no_header.csv', 
                 has_header=False, 
                 new_columns=column_names, 
                 try_parse_dates=True)
df.head()

invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
str,str,str,i64,str,i64,f64,str,date,str
"""I138884""","""C241288""","""Female""",28,"""Clothing""",5,1500.4,"""Credit Card""",2022-08-05,"""Kanyon"""
"""I317333""","""C111565""","""Male""",21,"""Shoes""",3,1800.51,"""Debit Card""",2021-12-12,"""Forum Istanbul…"
"""I127801""","""C266599""","""Male""",20,"""Clothing""",1,300.08,"""Cash""",2021-11-09,"""Metrocity"""
"""I173702""","""C988172""","""Female""",66,"""Shoes""",5,3000.85,"""Credit Card""",2021-05-16,"""Metropol AVM"""
"""I337046""","""C189076""","""Female""",53,"""Books""",4,60.6,"""Cash""",2021-10-24,"""Kanyon"""


In [35]:
df = pl.read_csv('../data/customer_shopping_data_no_header.csv', 
                 has_header=False, 
                 new_columns=column_names, 
                 try_parse_dates=True, 
                 dtypes={'age': pl.Int8, 'quantity': pl.Int32})
df.head()

invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
str,str,str,i8,str,i32,f64,str,date,str
"""I138884""","""C241288""","""Female""",28,"""Clothing""",5,1500.4,"""Credit Card""",2022-08-05,"""Kanyon"""
"""I317333""","""C111565""","""Male""",21,"""Shoes""",3,1800.51,"""Debit Card""",2021-12-12,"""Forum Istanbul…"
"""I127801""","""C266599""","""Male""",20,"""Clothing""",1,300.08,"""Cash""",2021-11-09,"""Metrocity"""
"""I173702""","""C988172""","""Female""",66,"""Shoes""",5,3000.85,"""Credit Card""",2021-05-16,"""Metropol AVM"""
"""I337046""","""C189076""","""Female""",53,"""Books""",4,60.6,"""Cash""",2021-10-24,"""Kanyon"""


In [43]:
df.write_csv('../data/output/shopping_data_output.csv', 
             has_header=False, 
             separator=',')

### There is more...

In [45]:
lf = pl.scan_csv('../data/customer_shopping_data_no_header.csv', 
                 has_header=False, 
                 new_columns=column_names, 
                 try_parse_dates=True, 
                 dtypes={'age': pl.Int8, 'quantity': pl.Int32})
lf.fetch(5)

invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
str,str,str,i8,str,i32,f64,str,date,str
"""I138884""","""C241288""","""Female""",28,"""Clothing""",5,1500.4,"""Credit Card""",2022-08-05,"""Kanyon"""
"""I317333""","""C111565""","""Male""",21,"""Shoes""",3,1800.51,"""Debit Card""",2021-12-12,"""Forum Istanbul…"
"""I127801""","""C266599""","""Male""",20,"""Clothing""",1,300.08,"""Cash""",2021-11-09,"""Metrocity"""
"""I173702""","""C988172""","""Female""",66,"""Shoes""",5,3000.85,"""Credit Card""",2021-05-16,"""Metropol AVM"""
"""I337046""","""C189076""","""Female""",53,"""Books""",4,60.6,"""Cash""",2021-10-24,"""Kanyon"""


In [47]:
lf.sink_csv('../data/output/shopping_data_output_sink.csv')

## Reading and writing parquet files

## Reading and writing JSON files

### Getting ready

### How to do it...

In [122]:
df = pl.read_json('../data/world_population.json')
df.select(df.columns[:10]).head()

place,pop1980,pop2000,pop2010,pop2022,pop2023,pop2030,pop2050,country,area
i64,f64,f64,f64,f64,i64,f64,f64,str,i64
356,6.96828385e8,1.0596e9,1.2406e9,1.4172e9,1428627663,1.5150e9,1.6705e9,"""India""",3287590
156,9.82372466e8,1.2641e9,1.3482e9,1.4259e9,1425671352,1.4156e9,1.3126e9,"""China""",9706961
840,2.23140018e8,2.82398554e8,3.11182845e8,3.38289857e8,339996563,3.52162301e8,3.75391963e8,"""United States""",9372610
360,1.48177096e8,2.14072421e8,2.44016173e8,2.75501339e8,277534122,2.921501e8,3.17225213e8,"""Indonesia""",1904569
586,8.0624057e7,1.54369924e8,1.94454498e8,2.35824862e8,240485658,2.74029836e8,3.67808468e8,"""Pakistan""",881912


In [102]:
df.write_json('../data/output/world_population_output.json')

In [121]:
df = pl.read_ndjson('../data/world_population.jsonl')
df.select(df.columns[:10]).head()

place,pop1980,pop2000,pop2010,pop2022,pop2023,pop2030,pop2050,country,area
i64,f32,f64,f64,f64,i64,f64,f64,str,i64
356,6.96828416e8,1.0596e9,1.2406e9,1.4172e9,1428627663,1.5150e9,1.6705e9,"""India""",3287590
156,9.8237248e8,1.2641e9,1.3482e9,1.4259e9,1425671352,1.4156e9,1.3126e9,"""China""",9706961
840,2.23140016e8,2.82398554e8,3.11182845e8,3.38289857e8,339996563,3.52162301e8,3.75391963e8,"""United States""",9372610
360,1.48177088e8,2.14072421e8,2.44016173e8,2.75501339e8,277534122,2.921501e8,3.17225213e8,"""Indonesia""",1904569
586,8.0624056e7,1.54369924e8,1.94454498e8,2.35824862e8,240485658,2.74029836e8,3.67808468e8,"""Pakistan""",881912


In [104]:
df.write_ndjson('../data/output/world_population_output.jsonl')

### There is more...

In [119]:
lf = pl.scan_ndjson('../data/world_population.jsonl')
lf.select(lf.columns[:10]).collect().head()

place,pop1980,pop2000,pop2010,pop2022,pop2023,pop2030,pop2050,country,area
i64,f64,f64,f64,f64,i64,f64,f64,str,i64
356,6.96828385e8,1.0596e9,1.2406e9,1.4172e9,1428627663,1.5150e9,1.6705e9,"""India""",3287590
156,9.82372466e8,1.2641e9,1.3482e9,1.4259e9,1425671352,1.4156e9,1.3126e9,"""China""",9706961
840,2.23140018e8,2.82398554e8,3.11182845e8,3.38289857e8,339996563,3.52162301e8,3.75391963e8,"""United States""",9372610
360,1.48177096e8,2.14072421e8,2.44016173e8,2.75501339e8,277534122,2.921501e8,3.17225213e8,"""Indonesia""",1904569
586,8.0624057e7,1.54369924e8,1.94454498e8,2.35824862e8,240485658,2.74029836e8,3.67808468e8,"""Pakistan""",881912


## Reading and writing CSV files

## Reading and writing CSV files

## Reading and writing CSV files

## Reading and writing CSV files